In [2]:
pip install scikit-learn

In [3]:
pip install nltk

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

mdata=pd.read_csv('/content/drive/MyDrive/review_dataset/TestReviews.csv')


Mounted at /content/drive


In [38]:
mdata.head()


,review,class
0,Fantastic spot for an even or a quite cocktail...,1
1,"Love, love, love the calamari. It's so good an...",1
2,"Love this place. Stiff martinis and cocktails,...",1
3,It's everything a great cocktail bar should be...,1
4,"I came here before a pirates game, so it was a...",1


In [8]:
ps=PorterStemmer()
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [9]:
corpus=[]
for i in range(0,len(mdata)):
  review=re.sub('[^a-zA-Z]',' ',mdata['review'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus.append(review)


In [27]:
cv=CountVectorizer(max_features=3000)
Tfidf=TfidfVectorizer(max_features=3000)
x=cv.fit_transform(corpus)
x1=Tfidf.fit_transform(corpus)
y=mdata.iloc[:,-1].values


In [29]:
lr=LogisticRegression(max_iter=900)
#lr.fit(x,y)
lr.fit(x1,y)

LogisticRegression(max_iter=900)

In [39]:
#n_review='This tastes awful,very bad service as well.'
n_review='Please remove veg vermicilli from the mess menu as it tastes awful.'
#n_review=re.sub('[^a-zA-Z]',' ',n_review)
#n_review=n_review.lower()
#n_review=n_review.split()
#n_review=[ps.stem(word) for word in n_review if not word in set(all_stopwords)]
#n_review=' '.join(n_review)
n_trans_review=cv.transform([n_review])
print(n_trans_review)
lr.predict(n_trans_review)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2 stored elements and shape (1, 3000)>
  Coords	Values
  (0, 1671)	1
  (0, 1674)	1


array([0])

In [41]:
def input_review():
  ch=True
  while(ch):
    n_review=input('Enter the review: ')
    n_trans_review=cv.transform([n_review])
    review=lr.predict(n_trans_review)
    print("Very glad you like our service. " if review==1 else print("Sorry for the bad service , gonna review about it soon. "))
    choice=input("Do you want to write more reviews (yes/no) : ")
    if choice=='no':
      ch=False
    else:
      ch=True
input_review()

Enter the review: the vermicilli taste really bad
Sorry for the bad service , gonna review about it soon. 
None
Do you want to write more reviews (yes/no) : no


In [31]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y,test_size=0.2,random_state=2)

In [33]:
x_lr=lr.fit(x_train,y_train)
x1_lr=lr.fit(x1_train,y1_train)


In [34]:
y_pred=x_lr.predict(x_test)
y1_pred=x1_lr.predict(x1_test)

In [37]:
print("Accuracy score using CountVectorizer ",accuracy_score(y_test,y_pred))
print("Accuracy score using TfidfVectorizer ",accuracy_score(y_test,y1_pred))

Accuracy score using CountVectorizer  0.8936416184971099
Accuracy score using TfidfVectorizer  0.923699421965318
